# HEP Plotter Gallery (1D)
Below are a collection of simple examples that demonstrate the capabilities of hepPlotter.  
This package is only meant to improve style and reduce time needed to make publication quality figures for HEP.  
Where possible, the API for matplotlib is used directly (the default options are modified to better suit HEP plots).  However, the user should still retain enough control to modify plots as needed.

First, ensure the environment has the proper path.

In [1]:
import os
import sys

In [2]:
cwd = os.getcwd()
hpd = cwd.rstrip("/hepPlotter/examples")
if hpd not in sys.path:
    sys.path.insert(0,hpd)
    print("Added {0} to path!".format(hpd))
else:
    print("Already exists in path!")

Added /Users/demarley/Desktop/CERN/CMS/common to path!


## One dimensional Histograms
hepPlotter is designed to accept ROOT histograms (TH1 or TEfficiency) and then plot in a HEP-friendly format using matplotlib.  It is also possible to pass un-binned data directly (e.g., a numpy array) and a unique histogram will be generated.  However, by using histograms directly, HEP users can manipulate histograms using familiar analysis techniques before plotting them.

In [3]:
from hepPlotter.hepPlotterHist1D import HepPlotterHist1D

import hepPlotter.labels as labels
import hepPlotter.tools as tools

Welcome to JupyROOT 6.10/02


/Users/demarley/anaconda2/lib/python2.7/site-packages/matplotlib/style/core.py:51: UserWarning: Style includes a parameter, 'backend', that is not related to style.  Ignoring
  "to style.  Ignoring".format(key))


In [4]:
variable_labels = labels.variable_labels()
sample_labels   = labels.sample_labels()

We will now load toy data (example.root) and show some basic plots that you can make using this framework.

In [5]:
import ROOT
# uproot integration is on-going

In [6]:
f = ROOT.TFile.Open("example.root")

In [7]:
gauss   = f.Get("gauss")
poisson = f.Get("poisson")

Compare two distributions.

- Ratio plot
- style (errorbar/histogram)
- legend
- add extra text
- labels
- data/mc (!!)

In [8]:
# set dummy values just to demonstrate the "labels.py" functionality
feature = "lep_pt"
sample1 = "ttbar"
sample2 = "wjets"

Basic plot showing two histograms

In [9]:
# Setup the plot (define some global settings)
hist = HepPlotterHist1D()

hist.normed  = True
hist.stacked = False
hist.x_label = variable_labels[feature].label
hist.y_label = "A.U." if hist.normed else "Events"
hist.saveAs  = "hist_"+feature
hist.CMSlabel = 'top left'
hist.CMSlabelStatus = "Simulation"

hist.initialize()   # sets the parameters of the plot

# Add 'possion' data to the plot from sample1
hist.Add(poisson,name="poisson",
         draw_type='step',color='red',edgecolor="red",
         label=sample_labels[sample1].label)

# Add 'gauss' data to the plot from sample2
hist.Add(gauss,name="gauss",
         draw_type='step',color='blue',edgecolor="blue",
         label=sample_labels[sample2].label)

p = hist.execute() # make the plot (p represents the matplotlib 'figure' object)
hist.savefig()     # save the figure (with name "hist.saveAs+hist.format") and close it

/Users/demarley/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:1328: UserWarning: findfont: Font family [u'sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [10]:
from IPython.display import IFrame
IFrame("hist_lep_pt.pdf", width=600, height=300)

Let's plot the same histograms but change the plotting style to demonstrate how to pass matplotlib plotting options directly through hepPlotter.

In [11]:
# Setup the plot
hist = HepPlotterHist1D()

hist.normed  = True
hist.stacked = False
hist.x_label = variable_labels[feature].label
hist.y_label = "A.U." if hist.normed else "Events"
hist.saveAs  = "hist_"+feature
hist.CMSlabel = 'top left'
hist.CMSlabelStatus = "Simulation"

hist.initialize()   # sets the parameters of the plot

# Add 'possion' data to the plot from sample1
hist.Add(poisson,name="poisson",
        draw_type='errorbar',color='red',edgecolor="red",label=sample_labels[sample1].label)

# Add 'gauss' data to the plot from sample2
hist.Add(gauss,name="gauss",
         draw_type='stepfilled',color='blue',alpha=0.5,
         edgecolor="blue",label=sample_labels[sample2].label)

p = hist.execute() # make the plot (p represents the matplotlib 'figure' object)
hist.savefig()     # save the figure (with name "hist.saveAs+hist.format") and close it

/Users/demarley/Desktop/CERN/CMS/common/hepPlotter/hepPlotterHist1D.py:81: RuntimeWarning: invalid value encountered in divide
  error = h_data.error * (data/og_data)  # scale error bars


In [12]:
IFrame("hist_lep_pt.pdf", width=600, height=300)

#### Ratio Plots

Now let's add a ratio plot to compare the two distributions.  
There are two options:
1. Ratio = A/B
2. Significance = A/sqrt(B)

(These can be modified or extended in the `HepPlotterHist1D` class.)  
You need to tell hepPlotter which data will be used for the ratio calculation (the numerator/denominator).
This can be achieved through the `ratio` attribute of `HepPlotterHist1D()`. 
This attribute represents the `HepPlotterRatio()` class, defined in `HepPlotterHist1D.py`,
that contains simple attribute for storing information about what kind of ratio plot to make and what properties it will have.

To add a subplot representing the ratio/significance of two distributions, you will need to add the following commands before `hist.execute()` is called:

```
hist.ratio.value  = "ratio"  # "significance"
hist.ratio.ylabel = "A/B"    # r"A/$\sqrt{\text{B}}$"
hist.ratio.Add(numerator='poisson',denominator='gauss',**kwargs)
```
where `**kwargs` represents a dictionary of matplotlib options that you can pass to hepPlotter.  Such a feature is necessary if you want to plot the ratio/significance with a different plotting style than the default style (the 'numerator').  
An example is shown below.

In [13]:
# Setup the plot
hist = HepPlotterHist1D()

hist.ratio_plot = "ratio"
hist.y_label_ratio = "A/B"
hist.normed  = True
hist.stacked = False
hist.x_label = variable_labels[feature].label
hist.y_label = "A.U." if hist.normed else "Events"
hist.saveAs  = "hist_"+feature
hist.CMSlabel = 'top left'
hist.CMSlabelStatus = "Simulation"

# Ratio plot -- see `HepPlotterRatio()` for all options
hist.ratio.value  = "significance"
hist.ratio.ylabel = "A/B"

hist.initialize()   # sets the parameters of the plot

# Add 'possion' data to the plot from sample1
hist.Add(poisson,name="poisson",
         draw_type='errorbar',color='red',edgecolor="red",
         label=sample_labels[sample1].label,ratios=[('gauss',True)]) # add 'ratios' keyword here

# Add 'gauss' data to the plot from sample2
hist.Add(gauss,name="gauss",
         draw_type='stepfilled',color='blue',alpha=0.5,
         edgecolor="blue",label=sample_labels[sample2].label)

# Add ratio plot
hist.ratio.Add(numerator='poisson',denominator='gauss')

p = hist.execute() # make the plot (p represents the matplotlib 'figure' object)
hist.savefig()     # save the figure (with name "hist.saveAs+hist.format") and close it

/Users/demarley/Desktop/CERN/CMS/common/hepPlotter/hepPlotterHist1D.py:224: RuntimeWarning: divide by zero encountered in divide
  ratio_data.data.content = (num_data / np.sqrt(den_data)).copy()
/Users/demarley/Desktop/CERN/CMS/common/hepPlotter/hepPlotterHist1D.py:224: RuntimeWarning: invalid value encountered in divide
  ratio_data.data.content = (num_data / np.sqrt(den_data)).copy()


In [14]:
IFrame("hist_lep_pt.pdf", width=600, height=300)

These two plots aren't too similar, so you can see that the ratio is only ~1 for a few bins.

Finally, let's add some extra text to the plot.  This can be useful when you need to add labels that describe an event selection, final state, or some other analysis-specific information.

In [19]:
# Setup the plot
hist = HepPlotterHist1D()

hist.ratio_plot = "ratio"
hist.y_label_ratio = "A/B"
hist.normed  = True
hist.stacked = False
hist.x_label = variable_labels[feature].label
hist.y_label = "A.U." if hist.normed else "Events"
hist.saveAs  = "hist_"+feature
hist.CMSlabel = 'top left'
hist.CMSlabelStatus = "Simulation"

# Add extra text onto the plot
# this is stored as a `PlotText()` object, defined in labels.py
# - kwargs are defined in `Text()` of labels.py
hist.extra_text.Add("Extra Text",coords=[0.03,0.80])

# Ratio plot -- see `HepPlotterRatio()` for all options
hist.ratio.value  = "significance"         # "ratio" 
hist.ratio.ylabel = r"A/$\sqrt{\text{B}}$" # "A/B"

hist.initialize()   # sets the parameters of the plot

# Add 'possion' data to the plot from sample1
hist.Add(poisson,name="poisson",
         draw_type='stepfilled',color='red',lw=0,alpha=0.5,
         edgecolor="red",label=sample_labels[sample1].label,
         ratios=[('gauss',True)]) # add 'ratios' keyword here

# Add 'gauss' data to the plot from sample2
hist.Add(gauss,name="gauss",
         draw_type='stepfilled',color='blue',alpha=0.5,
         edgecolor="blue",label=sample_labels[sample2].label)

# Add ratio plot -- modify some options to test the results
# NB: To switch plotting styles, use the keyword 'draw_type' just as in `hist.Add(...)`
hist.ratio.Add(numerator='poisson',denominator='gauss',lw=2,draw_type='errorbar')

p = hist.execute() # make the plot (p represents the matplotlib 'figure' object)
hist.savefig()     # save the figure (with name "hist.saveAs+hist.format") and close it

In [20]:
IFrame("hist_lep_pt.pdf", width=600, height=300)